In [1]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
more_features = False
use_staking = False
use_kernal_params = False
do_augment = True
fast = False

train_df = pd.read_csv('input/train' + ('_more_features' if more_features else '') + '.csv')
test_df = pd.read_csv('input/test'  + ('_more_features' if more_features else '') + '.csv')

do_lda = False

fix_data_skew = False

if fix_data_skew:
    trues = train_df.loc[train_df['target'] == 1]
    falses = train_df.loc[train_df['target'] != 1].sample(frac=1)[:len(trues)]
    train_df = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    train_df = train_df
    
X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [3]:
if do_lda:    
    lda = LDA(solver='svd', n_components=5, store_covariance=True)
    X_lda = pd.DataFrame(lda.fit_transform(X, y))
    X_test_lda = pd.DataFrame(lda.transform(X_test))
    X["lda"] = X_lda
    X_test["lda"] = X_test_lda

In [4]:
_, X_bottomhalf, _, y_bottomhalf = train_test_split(X, y, test_size=0.8, random_state=10)

In [5]:
if use_staking:
    X = X_bottomhalf
    y = y_bottomhalf

In [6]:
n_splits = 5
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [7]:
params = {
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'min_data_in_leaf': 2881,
    'max_depth': 0,
    'num_leaves': 3,
    'learning_rate': 0.01,
    'bagging_freq': 3,
    #'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 0.9),
    'feature_fraction': 0.8990901412442585,
    'bagging_seed': 11,
    'reg_alpha':  1.1173044727720816,
    'reg_lambda': 6.9285776442737514,
    'random_state': 42,
    'verbosity': -1,
    'subsample': 0.8054415526396443,
    'min_child_weight': 38.138072621096654,
    'num_threads': 4,
    'max_bin': 483
}

if use_kernal_params:
    # https://www.kaggle.com/jiweiliu/lgb-2-leaves-augment (lb: 0.901)
    params = {
        "objective" : "binary",
        "metric" : "auc",
        "boosting": 'gbdt',
        "max_depth" : -1,
        "num_leaves" : 13,
        "learning_rate" : 0.01,
        "bagging_freq": 5,
        "bagging_fraction" : 0.4,
        "feature_fraction" : 0.05,
        "min_data_in_leaf": 80,
        "min_sum_heassian_in_leaf": 10,
        "tree_learner": "serial",
        "boost_from_average": "false",
        #"lambda_l1" : 5,
        #"lambda_l2" : 5,
        "bagging_seed" : 13,
        "verbosity" : 1,
        "seed": 42
    }

In [8]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [9]:
oof = train_df[['ID_code', 'target']]
oof['predict'] = 0

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
predictions = test_df[['ID_code']]
val_aucs = []

In [11]:
score = 0.0
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    N = 5
    
    if not do_augment:
        N = 1
    
    p_valid,yp = 0,0
    for i in range(N):
        if do_augment:
            X_t, y_t = augment(X_train.values, y_train.values)
            X_t = pd.DataFrame(X_t)
            X_t = X_t.add_prefix('var_')

            trn_data = lgb.Dataset(X_t, label=y_t)
            val_data = lgb.Dataset(X_valid, label=y_valid)
        else:
            trn_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_valid, label=y_valid)
        
        evals_result = {}
        
        early_stopping_rounds = 800 if fast else 3000
        
        lgb_clf = lgb.train(params,
                        trn_data,
                        num_boost_round=2000000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=early_stopping_rounds,
                        verbose_eval = 300,
                        evals_result=evals_result
                       )
        p_valid += lgb_clf.predict(X_valid)
        yp += lgb_clf.predict(X_test)
    
    oof['predict'][valid_index] = p_valid/N
    val_score = roc_auc_score(y_valid, p_valid)
    val_aucs.append(val_score)
    
    predictions['fold{}'.format(fold_n+1)] = yp/N

Fold 0 started at Thu Mar 21 08:22:28 2019
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.762861	valid_1's auc: 0.754327
[600]	training's auc: 0.800331	valid_1's auc: 0.793664
[900]	training's auc: 0.821566	valid_1's auc: 0.814712
[1200]	training's auc: 0.835953	valid_1's auc: 0.828902
[1500]	training's auc: 0.84586	valid_1's auc: 0.838316
[1800]	training's auc: 0.8539	valid_1's auc: 0.845865
[2100]	training's auc: 0.860567	valid_1's auc: 0.852207
[2400]	training's auc: 0.865914	valid_1's auc: 0.857406
[2700]	training's auc: 0.870586	valid_1's auc: 0.86169
[3000]	training's auc: 0.874406	valid_1's auc: 0.86521
[3300]	training's auc: 0.877704	valid_1's auc: 0.868278
[3600]	training's auc: 0.880643	valid_1's auc: 0.870893
[3900]	training's auc: 0.883327	valid_1's auc: 0.873284
[4200]	training's auc: 0.8856	valid_1's auc: 0.875346
[4500]	training's auc: 0.887653	valid_1's auc: 0.877173
[4800]	training's auc: 0.889475	valid_1's auc: 0.878789
[5100]	

[8700]	training's auc: 0.902674	valid_1's auc: 0.890477
[9000]	training's auc: 0.903334	valid_1's auc: 0.891037
[9300]	training's auc: 0.903934	valid_1's auc: 0.891504
[9600]	training's auc: 0.904502	valid_1's auc: 0.891995
[9900]	training's auc: 0.90501	valid_1's auc: 0.892419
[10200]	training's auc: 0.905554	valid_1's auc: 0.892875
[10500]	training's auc: 0.905985	valid_1's auc: 0.893232
[10800]	training's auc: 0.906442	valid_1's auc: 0.893683
[11100]	training's auc: 0.906897	valid_1's auc: 0.894025
[11400]	training's auc: 0.907306	valid_1's auc: 0.894356
[11700]	training's auc: 0.907688	valid_1's auc: 0.894688
[12000]	training's auc: 0.908057	valid_1's auc: 0.894989
[12300]	training's auc: 0.908424	valid_1's auc: 0.895325
[12600]	training's auc: 0.908765	valid_1's auc: 0.895601
[12900]	training's auc: 0.909072	valid_1's auc: 0.895861
[13200]	training's auc: 0.909345	valid_1's auc: 0.89609
[13500]	training's auc: 0.909651	valid_1's auc: 0.896316
[13800]	training's auc: 0.909932	valid

[19800]	training's auc: 0.913694	valid_1's auc: 0.899329
[20100]	training's auc: 0.913827	valid_1's auc: 0.899409
[20400]	training's auc: 0.913958	valid_1's auc: 0.899508
[20700]	training's auc: 0.91408	valid_1's auc: 0.899559
[21000]	training's auc: 0.914201	valid_1's auc: 0.899614
[21300]	training's auc: 0.914326	valid_1's auc: 0.899671
[21600]	training's auc: 0.914441	valid_1's auc: 0.899714
[21900]	training's auc: 0.914565	valid_1's auc: 0.899791
[22200]	training's auc: 0.914687	valid_1's auc: 0.89987
[22500]	training's auc: 0.914804	valid_1's auc: 0.899915
[22800]	training's auc: 0.914916	valid_1's auc: 0.899939
[23100]	training's auc: 0.915017	valid_1's auc: 0.89996
[23400]	training's auc: 0.915122	valid_1's auc: 0.90001
[23700]	training's auc: 0.915231	valid_1's auc: 0.900052
[24000]	training's auc: 0.91533	valid_1's auc: 0.900048
[24300]	training's auc: 0.915428	valid_1's auc: 0.900084
[24600]	training's auc: 0.91554	valid_1's auc: 0.900134
[24900]	training's auc: 0.915637	vali

[30000]	training's auc: 0.917709	valid_1's auc: 0.899727
[30300]	training's auc: 0.917821	valid_1's auc: 0.899694
[30600]	training's auc: 0.917921	valid_1's auc: 0.899697
[30900]	training's auc: 0.918014	valid_1's auc: 0.899692
[31200]	training's auc: 0.918114	valid_1's auc: 0.899717
[31500]	training's auc: 0.918211	valid_1's auc: 0.899689
[31800]	training's auc: 0.918307	valid_1's auc: 0.899702
[32100]	training's auc: 0.918417	valid_1's auc: 0.899705
[32400]	training's auc: 0.918517	valid_1's auc: 0.899691
[32700]	training's auc: 0.91862	valid_1's auc: 0.899696
Early stopping, best iteration is:
[29964]	training's auc: 0.917696	valid_1's auc: 0.89973
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.761398	valid_1's auc: 0.753519
[600]	training's auc: 0.799785	valid_1's auc: 0.793776
[900]	training's auc: 0.82093	valid_1's auc: 0.814884
[1200]	training's auc: 0.834265	valid_1's auc: 0.828198
[1500]	training's auc: 0.844633	valid_1's auc: 0.838334


C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

...

Fold 4 started at Thu Mar 21 17:58:45 2019
Training until validation scores don't improve for 3000 rounds.
[300]	training's auc: 0.764662	valid_1's auc: 0.761483
[600]	training's auc: 0.802106	valid_1's auc: 0.794791
[900]	training's auc: 0.823472	valid_1's auc: 0.813948
[1200]	training's auc: 0.837128	valid_1's auc: 0.825838
[1500]	training's auc: 0.847503	valid_1's auc: 0.835012
[1800]	training's auc: 0.85535	valid_1's auc: 0.842104
[2100]	training's auc: 0.861664	valid_1's auc: 0.847904
[2400]	training's auc: 0.866942	valid_1's auc: 0.852577
[2700]	training's auc: 0.871226	valid_1's auc: 0.856617
[3000]	training's auc: 0.874868	valid_1's auc: 0.859928
[3300]	training's auc: 0.878384	valid_1's auc: 0.863014
[3600]	training's auc: 0.881182	valid_1's auc: 0.865535
[3900]	training's auc: 0.883677	valid_1's auc: 0.867929
[4200]	training's auc: 0.885941	valid_1's auc: 0.870052
[4500]	training's auc: 0.887949	valid_1's auc: 0.871935
[4800]	training's auc: 0.889741	valid_1's auc: 0.873658
[

[10200]	training's auc: 0.906117	valid_1's auc: 0.889361
[10500]	training's auc: 0.90659	valid_1's auc: 0.889775
[10800]	training's auc: 0.907063	valid_1's auc: 0.890191
[11100]	training's auc: 0.907479	valid_1's auc: 0.890531
[11400]	training's auc: 0.907884	valid_1's auc: 0.890886
[11700]	training's auc: 0.908288	valid_1's auc: 0.891223
[12000]	training's auc: 0.908633	valid_1's auc: 0.891539
[12300]	training's auc: 0.908972	valid_1's auc: 0.891813
[12600]	training's auc: 0.909338	valid_1's auc: 0.892101
[12900]	training's auc: 0.909683	valid_1's auc: 0.892373
[13200]	training's auc: 0.909966	valid_1's auc: 0.892629
[13500]	training's auc: 0.910281	valid_1's auc: 0.892874
[13800]	training's auc: 0.910564	valid_1's auc: 0.89311
[14100]	training's auc: 0.91083	valid_1's auc: 0.893301
[14400]	training's auc: 0.911074	valid_1's auc: 0.893461
[14700]	training's auc: 0.911316	valid_1's auc: 0.893693
[15000]	training's auc: 0.911544	valid_1's auc: 0.893923
[15300]	training's auc: 0.911796	v

[18900]	training's auc: 0.912975	valid_1's auc: 0.895569
[19200]	training's auc: 0.913108	valid_1's auc: 0.895647
[19500]	training's auc: 0.913256	valid_1's auc: 0.895749
[19800]	training's auc: 0.913392	valid_1's auc: 0.895819
[20100]	training's auc: 0.913526	valid_1's auc: 0.895904
[20400]	training's auc: 0.913658	valid_1's auc: 0.895953
[20700]	training's auc: 0.913777	valid_1's auc: 0.896018
[21000]	training's auc: 0.913916	valid_1's auc: 0.896078
[21300]	training's auc: 0.91405	valid_1's auc: 0.896149
[21600]	training's auc: 0.914177	valid_1's auc: 0.896207
[21900]	training's auc: 0.914288	valid_1's auc: 0.896255
[22200]	training's auc: 0.914407	valid_1's auc: 0.896344
[22500]	training's auc: 0.914531	valid_1's auc: 0.896397
[22800]	training's auc: 0.914645	valid_1's auc: 0.896451
[23100]	training's auc: 0.914758	valid_1's auc: 0.89652
[23400]	training's auc: 0.91487	valid_1's auc: 0.89656
[23700]	training's auc: 0.914977	valid_1's auc: 0.896617
[24000]	training's auc: 0.915074	va

[27600]	training's auc: 0.917138	valid_1's auc: 0.896966
[27900]	training's auc: 0.917243	valid_1's auc: 0.896988
[28200]	training's auc: 0.917345	valid_1's auc: 0.897037
[28500]	training's auc: 0.917446	valid_1's auc: 0.897029
[28800]	training's auc: 0.917538	valid_1's auc: 0.897036
[29100]	training's auc: 0.91763	valid_1's auc: 0.897033
[29400]	training's auc: 0.917737	valid_1's auc: 0.897065
[29700]	training's auc: 0.917822	valid_1's auc: 0.89709
[30000]	training's auc: 0.917924	valid_1's auc: 0.897105
[30300]	training's auc: 0.918024	valid_1's auc: 0.897099
[30600]	training's auc: 0.918112	valid_1's auc: 0.897084
[30900]	training's auc: 0.918209	valid_1's auc: 0.897092
[31200]	training's auc: 0.9183	valid_1's auc: 0.89709
[31500]	training's auc: 0.918395	valid_1's auc: 0.897116
[31800]	training's auc: 0.918484	valid_1's auc: 0.897123
[32100]	training's auc: 0.918585	valid_1's auc: 0.897114
[32400]	training's auc: 0.918675	valid_1's auc: 0.897128
[32700]	training's auc: 0.918781	val

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\yuzos\Anaconda3\lib\site-packages\pandas\core\series.py:977: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._set_labels(key, value)
C:\Users\yuzos\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_l

In [12]:
mean_auc = np.mean(val_aucs)
std_auc = np.std(val_aucs)
all_auc = roc_auc_score(oof['target'], oof['predict'])
print("Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

Mean auc: 0.901320857, std: 0.003065016. All auc: 0.901319637.


In [13]:
# submission
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)
predictions.to_csv('lgb_all_predictions.csv', index=None)
sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df["target"] = predictions['target']
sub_df.to_csv("lgb_submission.csv", index=False)
oof.to_csv('lgb_oof.csv', index=False)

C:\Users\yuzos\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
